<a href="https://colab.research.google.com/github/dmydud/theory-for-data-science/blob/main/LinAlgAnim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !sudo apt update
# !sudo apt install libcairo2-dev ffmpeg \
#     texlive texlive-latex-extra texlive-fonts-extra \
#     texlive-latex-recommended texlive-science \
#     tipa libpango1.0-dev
# !pip install manim
# !pip install IPython==8.21.0

In [47]:
from IPython.display import HTML
from base64 import b64encode
import os
from copy import deepcopy

import numpy as np
from manim import *

config['quality'] = "medium_quality"
config['verbosity'] = "WARNING"

PATH_TO_VIDEO = "media/videos/720p30/"


def mp4_play(name, width=720, dirpath=PATH_TO_VIDEO):
    if isinstance(name, Scene):
        name = name.__class__.__name__
    elif not isinstance(name, str):
        name = name.__name__
    _mp4 = open(os.path.join(dirpath,  name + '.mp4'),'rb').read()
    _data_url = "data:video/mp4;base64," + b64encode(_mp4).decode()
    return HTML(f"""
    <video width={width} controls>
        <source src="{_data_url}" type="video/mp4">
    </video>
    """)

## 1. The Geometry of Linear Equations

The fundamental problem of linear algebra is to solve a system of linear equations.

### **Ex. 1**
$$\begin{cases}2x - y = 0 \\ -x + 2y = 3\end{cases}$$

This system of equations can be written in matrix form as:
$$\begin{bmatrix}2 & -1 \\ -1 & 2\end{bmatrix}\begin{bmatrix}x \\ y\end{bmatrix} = \begin{bmatrix}0 \\ 3\end{bmatrix}$$

Thus, the matrix equation is:
$$A\mathbf{x} = \mathbf{b}$$

where $A$ is the coefficient matrix, $\mathbf{x}$ is the vector of unknowns, and $\mathbf{b}$ is the column vector.


In [70]:
#@title Row picture
class RowPicture(Scene):
    def construct(self):
        plane = NumberPlane(
            x_range=(-5, 5, 1),
            y_range=(-4, 4, 1),
            x_length=7,
            y_length=5.6,
        ).move_to(RIGHT*3)

        sys_eqs = Tex(r"$\begin{cases} 2x - y = 0 \\ -x + 2y = 3 \end{cases}$")
        sys_eqs.move_to(2*UP + LEFT*4)

        sys_eq1_pos = sys_eqs.get_center() + UP*sys_eqs.height/4
        sys_eq2_pos = sys_eqs.get_center() - UP*sys_eqs.height/4


        self.play(AnimationGroup(Create(plane), Write(sys_eqs), lag_ratio=.7), run_time=3)

        def create_and_animate_eq(eq_str, color_idxs, start_pos=None, transform_run_time=1):
            eq = Tex(*eq_str)
            for color_idx in color_idxs:
                eq[color_idx].set_color(YELLOW)
            if start_pos is None:
                eq.move_to(LEFT*4)
                self.play(Write(eq))
                return eq

            new_eq = deepcopy(eq)
            eq.move_to(start_pos)
            new_eq.move_to(LEFT*4)
            self.play(AnimationGroup(FadeOut(eq), Transform(eq, new_eq), lag_ration=0), run_time=transform_run_time)
            return new_eq

        def transform_eq(old_eq, new_eq_str, color_idxs, run_time=1):
            new_eq = Tex(*new_eq_str)
            for color_idx in color_idxs:
                new_eq[color_idx].set_color(YELLOW)
            new_eq.move_to(LEFT*4)
            self.play(FadeOut(old_eq), Transform(old_eq, new_eq), run_time=run_time)
            return new_eq

        def create_point_and_label(eq, point, label, pos):
            dot = Dot(plane.c2p(*point), color=YELLOW)
            self.play(AnimationGroup(FadeOut(eq), Transform(eq, dot), Create(dot), lag_ratio=0), run_time=1)
            label = Tex(f"({point[0]}, {point[1]})").set_color(YELLOW)
            label.next_to(dot, pos)
            self.play(Write(label), run_time=1.5)
            return dot, label

        def create_line(slope, intercept, x_range=(-5, 5), y_range=(-4, 4)):
            line_func = lambda x: slope * x + intercept

            x_min, x_max = x_range
            y_min, y_max = y_range

            points = [
                plane.c2p(x, line_func(x))
                for x in np.linspace(x_min, x_max, 100)
                if y_min <= line_func(x) <= y_max
            ]

            line = VMobject()
            line.set_points_as_corners(points)
            line.set_color(YELLOW)

            self.play(Create(line))

            return line

        def deactivate_line(line, point1, point2, label1, label2):
            self.play(AnimationGroup(FadeOut(label1), FadeOut(label2), FadeOut(point1), FadeOut(point2), lag_ratio=0))
            line.set_color(WHITE)

        eq = create_and_animate_eq([r"$2$", r"$x$", r"$\ -\ $", r"$y$", r"$\ = 0$"], [1, 3], start_pos=sys_eq1_pos)
        eq = transform_eq(eq, [r"$2$", r"$(0)$", r"$\ -\ $", r"$y$", r"$\ = 0$"], [1, 3])
        eq = transform_eq(eq, [r"$0$", r"$\ -\ $", r"$y$", r"$\ = 0$"], [2])
        eq = transform_eq(eq, [r"$y$", r"$\ = 0$"], [0], run_time=1.5)
        eq1_point1, eq1_label1 = create_point_and_label(eq, (0, 0), "(0, 0)", .5*DOWN + .5*RIGHT)

        eq = create_and_animate_eq([r"$2$", r"$x$", r"$\ -\ $", r"$y$", r"$\ = 0$"], [1, 3], start_pos=sys_eq1_pos)
        eq = transform_eq(eq, [r"$2$", r"$(1)$", r"$\ -\ $", r"$y$", r"$\ = 0$"], [1, 3])
        eq = transform_eq(eq, [r"$2$", r"$\ -\ $", r"$y$", r"$\ = 0$"], [2])
        eq = transform_eq(eq, [r"$y$", r"$\ = 2$"], [0], run_time=1.5)
        eq1_point2, eq1_label2 = create_point_and_label(eq, (1, 2), "(1, 2)", .5*DOWN + .5*RIGHT)

        slope = 2
        line1 = create_line(slope, 0)
        line1_label = Tex(r"$2x-y=0$")
        line1_label.move_to(2.35*RIGHT+2*DOWN).rotate(np.arctan(slope))

        deactivate_line(line1, eq1_point1, eq1_point2, eq1_label1, eq1_label2)
        self.play(Write(line1_label))

        eq = create_and_animate_eq([r"$-$", r"$x$", r"$\ +2\ $", r"$y$", r"$\ = 3$"], [1, 3], start_pos=sys_eq2_pos)
        eq = transform_eq(eq, [r"$-$", r"$x$", r"$\ +2\ $", r"$(0)$", r"$\ = 3$"], [1, 3])
        eq = transform_eq(eq, [r"$-$", r"$x$", r"$\ +\ $", r"$0$", r"$\ = 3$"], [1, 3])
        eq = transform_eq(eq, [r"$-$", r"$x$", r"$\ = 3$"], [1])
        eq = transform_eq(eq, [r"$x$", r"$\ = -3$"], [0])
        eq2_point1, eq2_label1 = create_point_and_label(eq, (-3, 0), "(-3, 0)", .5*UP + .5*LEFT)

        eq = create_and_animate_eq([r"$-$", r"$x$", r"$\ +2\ $", r"$y$", r"$\ = 3$"], [1, 3], start_pos=sys_eq2_pos)
        eq = transform_eq(eq, [r"$-$", r"$( -1)$", r"$\ +2\ $", r"$y$", r"$\ = 3$"], [1, 3])
        eq = transform_eq(eq, [r"$1$", r"$\ +2\ $", r"$y$", r"$\ = 3$"], [2])
        eq = transform_eq(eq, [r"$2$", r"$y$", r"$\ = 2$"], [1])
        eq = transform_eq(eq, [r"$y$", r"$\ = 1$"], [0])
        eq2_point2, eq2_label2 = create_point_and_label(eq, (-1, 1), "(-1, 1)", .5*UP + .5*LEFT)

        slope = .5
        line2 = create_line(slope, 1.5)
        line2_label = Tex(r"$-x+2y=3$")
        line2_label.move_to(1.5*RIGHT+.9*UP).rotate(np.arctan(slope))

        deactivate_line(line2, eq2_point1, eq2_point2, eq2_label1, eq2_label2)
        self.play(Write(line2_label))

        self.wait(.3)

        self.add(eq1_point2, eq1_label2)

        root = Tex(r"$\begin{cases} x = 1 \\ y = 2 \end{cases}$")
        root.move_to(LEFT*4)

        self.play(Transform(eq1_label2, root))

        self.wait()

_scene = RowPicture()
_scene.render()
print("Row picture:")
mp4_play(_scene)

Row picture:
